# SETUP
The following blocks are to be executed first of anything else, to setup import, classes, functions and 
constants that are needed for all stages

In [1]:
import os
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import resnet34

from PIL import Image
from tqdm import tqdm

from logs import Logger, generate_model_checkpoint_name, generate_log_filenames

from gtea_dataset import GTEA61, GTEA61_flow, GTEA61_2Stream
from attention_model_MS_reg import attention_model_ms_reg
from flow_resnet import flow_resnet34
from twoStreamModel import twoStreamAttentionModel
from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip, DownSampling, To1Dimension)

In [2]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
NUM_CLASSES = 61 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 32     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

MMAP_LENGTH = 49
DATA_DIR = '../GTEA61'
model_folder = '../saved_models'

In [3]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
default_transforms = [
    Scale(256),
    RandomHorizontalFlip(),
    MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
    ToTensor()
]

spatial_transform = Compose(default_transforms + [normalize])
spatial_transform_mmaps = Compose(default_transforms + [DownSampling(), To1Dimension()])
spatial_transform_val = Compose([Scale(256), CenterCrop(224), ToTensor(), normalize])

# Stage 1 specific-setup

In [15]:
STAGE = 1

LR = 0.001                            # The initial Learning Rate
MOMENTUM = 0.9                        # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 4e-5                   # Regularization, you can keep this at the default

NUM_EPOCHS = 200                      # Total number of training epochs (iterations over dataset)
STEP_SIZE = [25, 75, 150]             # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1                           # Multiplicative factor for learning rate step-down
MEM_SIZE = 512
SEQ_LEN = 7

# this dictionary is needed for the logger class
parameters = {
    'DEVICE': DEVICE,
    'NUM_CLASSES': NUM_CLASSES,
    'BATCH_SIZE': BATCH_SIZE,
    'LR': LR,
    'MOMENTUM': MOMENTUM,
    'WEIGHT_DECAY': WEIGHT_DECAY,
    'NUM_EPOCHS': NUM_EPOCHS,
    'STEP_SIZE': STEP_SIZE,
    'GAMMA': GAMMA,
    'MEM_SIZE': MEM_SIZE,
    'SEQ_LEN': SEQ_LEN
}

In [16]:
# Prepare Pytorch train/test Datasets
train_dataset = GTEA61(DATA_DIR, split = 'train', transform = spatial_transform, seq_len = SEQ_LEN,
                       mmaps = True, mmaps_transform = spatial_transform_mmaps)
test_dataset = GTEA61(DATA_DIR, split = 'test', transform = spatial_transform_val, seq_len = SEQ_LEN)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Train Dataset: 341
Test Dataset: 116


In [17]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4, drop_last = True)
val_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)

# Prepare stage 1

In [53]:
validate = True

model = attention_model_ms_reg(num_classes = NUM_CLASSES, mem_size = MEM_SIZE)
model.train(False)

for params in model.parameters():
    params.requires_grad = False

for params in model.lstm_cell.parameters():
    params.requires_grad = True

for params in model.classifier.parameters():
    params.requires_grad = True

model.lstm_cell.train(True)
model.classifier.train(True)
model.to(DEVICE)

trainable_params = [p for p in model.parameters() if p.requires_grad]

loss_fn = nn.CrossEntropyLoss()

optimizer_fn = optim.Adam(trainable_params, lr = LR, weight_decay = WEIGHT_DECAY, eps = 1e-4)

optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones = STEP_SIZE, gamma = GAMMA)

# Stage 1

In [ ]:
train_iter = 0
val_iter = 0
min_accuracy = 0

trainSamples = len(train_dataset) - (len(train_dataset) % BATCH_SIZE)
val_samples = len(test_dataset) 
iterPerEpoch = len(train_loader)
val_steps = len(val_loader)

cudnn.benchmark

train_log, val_log = generate_log_filenames(STAGE, SEQ_LEN, ms_block = True)
model_checkpoint = generate_model_checkpoint_name(STAGE, SEQ_LEN, ms_block = True)

train_log_file = os.path.join(model_folder, train_log)
val_log_file = os.path.join(model_folder, val_log)

train_logger = Logger(**parameters)
val_logger = Logger(**parameters)

for epoch in range(NUM_EPOCHS):

    epoch_loss = 0
    numCorrTrain = 0
    
    model.lstm_cell.train(True)
    model.classifier.train(True)
        
    for i, (inputs, _, targets) in enumerate(train_loader):

        train_iter += 1

        optimizer_fn.zero_grad()

        inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
        labelVariable = targets.to(DEVICE)
        output_label, _ = model(inputVariable)
        
        loss = loss_fn(output_label, labelVariable)
        loss.backward()
        optimizer_fn.step()
            
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += torch.sum(predicted == labelVariable.data).data.item()

        step_loss = loss.data.item()
        epoch_loss += step_loss

        train_logger.add_step_data(train_iter, numCorrTrain, step_loss)
    
    avg_loss = epoch_loss/iterPerEpoch

    trainAccuracy = (numCorrTrain / trainSamples) * 100
    train_logger.add_epoch_data(epoch+1, trainAccuracy, avg_loss)
    print('Train: Epoch = {} | Loss = {:.3f} | Accuracy = {:.3f}'.format(epoch+1, avg_loss, trainAccuracy))

    if validate:

        if (epoch+1) % 1 == 0:

            model.train(False)

            val_loss_epoch = 0
            numCorr = 0

            for j, (inputs, targets) in enumerate(val_loader):

                val_iter += 1

                inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
                labelVariable = targets.to(DEVICE)
                
                output_label, _ = model(inputVariable)

                val_loss = loss_fn(output_label, labelVariable)
                val_loss_step = val_loss.data.item()
                val_loss_epoch += val_loss_step
                
                _, predicted = torch.max(output_label.data, 1)
                numCorr += torch.sum(predicted == labelVariable.data).data.item()

                val_logger.add_step_data(val_iter, numCorr, val_loss_step)
                
            val_accuracy = (numCorr / val_samples) * 100
            avg_val_loss = val_loss_epoch / val_steps
            val_logger.add_epoch_data(epoch+1, val_accuracy, avg_val_loss)

            print('Val: Epoch = {} | Loss {:.3f} | Accuracy = {:.3f}'.format(epoch + 1, avg_val_loss, val_accuracy))

            if val_accuracy > min_accuracy:
                print("[||| NEW BEST on val||||]")
                save_path_model = os.path.join(model_folder, model_checkpoint)
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
        
    train_logger.save(train_log_file)
    val_logger.save(val_log_file)
    optim_scheduler.step()

# Stage 2 specific setup

In [8]:
STAGE = 2

LR = 0.0001            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 4e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 150      # Total number of training epochs (iterations over dataset)
STEP_SIZE = [25, 75] # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
MEM_SIZE = 512
SEQ_LEN = 7

parameters = {
    'DEVICE': DEVICE,
    'NUM_CLASSES': NUM_CLASSES,
    'BATCH_SIZE': BATCH_SIZE,
    'LR': LR,
    'MOMENTUM': MOMENTUM,
    'WEIGHT_DECAY': WEIGHT_DECAY,
    'NUM_EPOCHS': NUM_EPOCHS,
    'STEP_SIZE': STEP_SIZE,
    'GAMMA': GAMMA,
    'MEM_SIZE': MEM_SIZE,
    'SEQ_LEN': SEQ_LEN
}

In [9]:
# Prepare Pytorch train/test Datasets
train_dataset = GTEA61(DATA_DIR, split='train', transform=spatial_transform,
                       seq_len=SEQ_LEN, mmaps = True, mmaps_transform = spatial_transform_mmaps)
test_dataset = GTEA61(DATA_DIR, split='test', transform=spatial_transform_val,
                      seq_len=SEQ_LEN)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Train Dataset: 341
Test Dataset: 116


In [10]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Prepare stage 2

In [30]:
best_old_stage = generate_model_checkpoint_name(stage=1, n_frames=SEQ_LEN, ms_block = True)
stage1_dict = os.path.join(model_folder, best_old_stage)
validate = True

model = attention_model_ms_reg(num_classes=NUM_CLASSES, mem_size=MEM_SIZE)
model.load_state_dict(torch.load(stage1_dict))

model.train(False)

for params in model.parameters():
    params.requires_grad = False

layers_to_train = [
    model.resNet.layer4[0].conv1,
    model.resNet.layer4[0].conv2,
    model.resNet.layer4[1].conv1,
    model.resNet.layer4[1].conv2,
    model.resNet.layer4[2].conv1,
    model.resNet.layer4[2].conv2,
    model.resNet.fc,
    model.lstm_cell,
    model.classifier,
    model.msBlock
]

for layer in layers_to_train:
    for params in layer.parameters():
        params.requires_grad = True

for layer in layers_to_train:
    layer.train(True)

model.to(DEVICE)

loss_class = nn.CrossEntropyLoss()
loss_reg = nn.MSELoss(reduction='sum')

trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer_fn = torch.optim.Adam(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-4)

optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=STEP_SIZE, gamma=GAMMA)

# Stage 2

In [31]:
train_iter = 0
val_iter = 0
min_accuracy = 0

trainSamples = len(train_dataset) - (len(train_dataset) % BATCH_SIZE)
val_samples = len(test_dataset)

iterPerEpoch = len(train_loader)
val_steps = len(val_loader)

cudnn.benchmark

train_log, val_log = generate_log_filenames(STAGE, SEQ_LEN, mmaps=True, optional="_reg")
model_checkpoint = generate_model_checkpoint_name(STAGE, SEQ_LEN, mmaps=True, optional="_reg"))

train_log_file = os.path.join(model_folder, train_log)
val_log_file = os.path.join(model_folder, val_log)
train_logger_2 = Logger(**parameters)
val_logger_2 = Logger(**parameters)

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    numCorrTrain = 0
    
    for layer in layers_to_train:
        layer.train(True)
    
    for i, (inputs, mmaps, targets) in enumerate(train_loader):
        train_iter += 1

        optimizer_fn.zero_grad()

        inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
        labelVariable = targets.to(DEVICE)
        mmaps = mmaps.permute(1, 0, 2).to(DEVICE)

        output_label, _, output_label_mmaps = model(inputVariable, mmaps = True)
        
        loss_rgb = loss_class(output_label, labelVariable)
        loss_mmaps = loss_reg(output_label_mmaps, mmaps)/(SEQ_LEN*BATCH_SIZE)
    
        loss = loss_rgb + loss_mmaps
        loss.backward()

        optimizer_fn.step()
        
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += torch.sum(predicted == labelVariable.data).data.item()
        step_loss = loss.data.item()
        epoch_loss += step_loss
        train_logger_2.add_step_data(train_iter, numCorrTrain, step_loss)

        
    avg_loss = epoch_loss/iterPerEpoch
    trainAccuracy = (numCorrTrain / trainSamples) * 100
    train_logger_2.add_epoch_data(epoch+1, trainAccuracy, avg_loss)
    
    print('Train: Epoch = {} | Loss = {:.3f} | Accuracy = {:.3f}'.format(epoch+1, avg_loss, trainAccuracy))

    if validate is not None:
        if (epoch+1) % 1 == 0:
            model.train(False)
            val_loss_epoch = 0
            numCorr = 0
            for j, (inputs, targets) in enumerate(val_loader):
                val_iter += 1

                inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
                labelVariable = targets.to(DEVICE)
                
                output_label, _ = model(inputVariable)

                val_loss = loss_fn(output_label, labelVariable)
                val_loss_step = val_loss.data.item()
                val_loss_epoch += val_loss_step

                _, predicted = torch.max(output_label.data, 1)

                numCorr += torch.sum(predicted == labelVariable.data).data.item()
                val_logger_2.add_step_data(val_iter, numCorr, val_loss_step)

            val_accuracy = (numCorr / val_samples) * 100
            avg_val_loss = val_loss_epoch / val_steps

            print('Val: Epoch = {} | Loss {:.3f} | Accuracy = {:.3f}'.format(epoch + 1, avg_val_loss, val_accuracy))
            
            val_logger_2.add_epoch_data(epoch+1, val_accuracy, avg_val_loss)
            
            if val_accuracy > min_accuracy:
                print("[||| NEW BEST on val |||]")
                save_path_model = os.path.join(model_folder, model_checkpoint)
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
            
    optim_scheduler.step()

    train_logger_2.save(train_log_file)
    val_logger_2.save(val_log_file)

Train: Epoch = 1 | Loss = 9.521 | Accuracy = 2.812
Val: Epoch = 1 | Loss 4.372 | Accuracy = 1.724
[||| NEW BEST on val |||]
Train: Epoch = 2 | Loss = 6.913 | Accuracy = 1.875
Val: Epoch = 2 | Loss 4.188 | Accuracy = 1.724
Train: Epoch = 3 | Loss = 6.577 | Accuracy = 2.500
Val: Epoch = 3 | Loss 4.130 | Accuracy = 0.862
Train: Epoch = 4 | Loss = 6.409 | Accuracy = 3.750
Val: Epoch = 4 | Loss 4.041 | Accuracy = 3.448
[||| NEW BEST on val |||]
Train: Epoch = 5 | Loss = 6.502 | Accuracy = 4.688
Val: Epoch = 5 | Loss 4.020 | Accuracy = 5.172
[||| NEW BEST on val |||]
Train: Epoch = 6 | Loss = 6.404 | Accuracy = 4.688
Val: Epoch = 6 | Loss 3.976 | Accuracy = 6.034
[||| NEW BEST on val |||]
Train: Epoch = 7 | Loss = 6.294 | Accuracy = 8.125
Val: Epoch = 7 | Loss 3.966 | Accuracy = 6.034
Train: Epoch = 8 | Loss = 6.447 | Accuracy = 7.812
Val: Epoch = 8 | Loss 3.921 | Accuracy = 6.897
[||| NEW BEST on val |||]
Train: Epoch = 9 | Loss = 6.279 | Accuracy = 6.562
Val: Epoch = 9 | Loss 3.907 | Accur

KeyboardInterrupt: 